## Single run example of nanotribological properties workflow

### Background
- Using SAM as coating to reduce coefficient of friction and force of adhesion when shearing between two surface
- Many variables that can be varied and, hence, makes it a prime candidate for Computational Simulation
- Several MD studies focusing on different properties
    - Andrew Paper: backbone chain length, chain density, terminal groups
    - Current screening project: mixed composition 
    - Near future project: backbone chemitries
- The legacy encourage this research setup to be TRUE
    - Future research can continue this project by varing and/or adding more variables based on the original project
    
### Model
- Initialization of two surfaces coated with SAM - varying mixing composition, backbone chainlength, terminal groups
- Two surface was then energy minimized through a few steps (by LAMMPS and GROMACS)
- The two system is then compressed and sheared against each other (the top surface moving, and the bottom surface fixed)
- The output is analyzed with mdanalysis

### Simulation Details
- Initialze by MoSDeF
- Fix overlaps by LAMMPS
- Energy Minimize by GROMACS
- NVT Equilibrate by GROMACS
- Compress by GROMACS
- Shearing at 5nN, 15nN, 25nN by GROMACS
- Data Analysis with mdanalysis

#### System initialization

In [20]:
# First change into working directory
%cd example_simulation

/Users/coquach/Documents/TRUE/TRUE-nanotribology/workflow/single_run/example_simulation


In [2]:
import flow
import numpy as np
import scipy
from foyer import Forcefield
from mbuild.formats.lammpsdata import write_lammpsdata
from mbuild.lib.atoms import H
from ctools.fileio import write_monolayer_ndx, read_ndx
from ctools.lib.chains import Alkylsilane
from ctools.lib.recipes import DualSurface, SilicaInterface, SurfaceMonolayer
from util.index_groups import generate_index_groups

In [5]:
""" Define system variable"""
chainlength = 17
backbone_A = Alkylsilane
backbone_B = Alkylsilane
seed = 27
pattern_type = "random"
terminal_group = "methyl"
num_chains = 100
"""
-----------------------------------
Generate amorphous silica interface
-----------------------------------
"""
surface_a = SilicaInterface(thickness=1.2, seed=seed)
surface_b = SilicaInterface(thickness=1.2, seed=seed)

"""
------------------------------------------------------
Generate prototype of functionalized alkylsilane chain
------------------------------------------------------
"""
chain_prototype_A = backbone_A(
    chain_length=chainlength, terminal_group=terminal_group)
chain_prototype_B = backbone_B(
    chain_length=chainlength, terminal_group=terminal_group)
"""
----------------------------------------------------------
Create monolayer on surface, backfilled with hydrogen caps
----------------------------------------------------------
"""
# bottom monolayer is backfilled with the other terminal group
# num_chains = num_chains * a_fraction
monolayer_a = SurfaceMonolayer(
    surface=surface_a,
    chains=chain_prototype_A,
    n_chains=num_chains,
    seed=seed,
    backfill=H(),
    rotate=False,
)
monolayer_a.name = "Bottom"
monolayer_b = SurfaceMonolayer(
    surface=surface_b,
    chains=chain_prototype_B,
    n_chains=num_chains,
    seed=seed,
    backfill=H(),
    rotate=False,
)
monolayer_b.name = "Top"

"""
----------------------
Create dual monolayers
----------------------
"""
dual_monolayer = DualSurface(
    bottom=monolayer_a, top=monolayer_b, separation=2.0
)

"""
--------------------------------------------------------
Make sure box is elongated in z to be pseudo-2D periodic
--------------------------------------------------------
"""
box = dual_monolayer.boundingbox
dual_monolayer.periodicity += np.array([0, 0, 5.0 * box.lengths[2]])

"""
-------------------------------------------------------------------
- Save to .GRO, .TOP, and .LAMMPS formats
- Atom-type the system using Foyer, with parameters from the OPLS
force field obtained from GROMACS. Parameters are located in a
Foyer XML file in the `atools` git repo, with references provided
as well as notes where parameters have been added or altered to
reflect the literature.
-------------------------------------------------------------------
"""

forcefield_filepath = "./util/forcefield/oplsaa.xml"

dual_monolayer.save("init.gro", residues=["Top", "Bottom"], overwrite=True)

structure = dual_monolayer.to_parmed(
    box=None, residues=["Top", "Bottom"]
    )
ff = Forcefield(forcefield_files=forcefield_filepath)
structure = ff.apply(structure)
structure.combining_rule = "geometric"

structure.save("init.top", overwrite=True)
write_lammpsdata(filename="init.lammps", structure=structure)

"""
--------------------------------------
Specify index groups and write to file
--------------------------------------
"""
index_groups = generate_index_groups(
    system=dual_monolayer,
    terminal_group=terminal_group,
    freeze_thickness=0.5,
)
write_monolayer_ndx(rigid_groups=index_groups, filename="./example_simulation/init.ndx")

/Users/coquach/Documents/conda_env/mosdef36/mbuild/mbuild/utils/io.py:120: DeprecationWarning: openbabel 2.0 detected and will be dropped in a future release. Consider upgrading to 3.x.
  warnings.warn(msg, DeprecationWarning)
/Users/coquach/miniconda3/envs/mosdef36/lib/python3.6/site-packages/mdtraj/formats/mol2.py:171: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col=0, header=None, sep="\s*", engine='python')
/Users/coquach/miniconda3/envs/mosdef36/lib/python3.6/site-packages/pandas/io/parsers.py:2379: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/coquach/miniconda3/envs/mosdef36/lib/python3.6/site-packages/pandas/io/parsers.py:2381: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/coquach/Documents/conda_env/mosdef36/mbuild/mbuild/utils/io.py:120: DeprecationWarning: openbabel 2.0 detected and will be dropped in a future release. Consider upgrading to 3.x.


No urey bradley terms detected, will use angle_style harmonic
RB Torsions detected, will use dihedral_style opls
bottom_frozen: 636
top_frozen: 636
bottom_surface: 2132
top_surface: 2132
surfaces: 4264
bottom_chains: 6000
top_chains: 6000
chains: 12000
bottom_termini: 400
top_termini: 400
terminal_groups: 800
bottom: 8132
top: 8132
System: 16264


In [6]:
dual_monolayer.visualize()

/Users/coquach/Documents/conda_env/mosdef36/mbuild/mbuild/compound.py:2410: UserWarning: Guessing that "<OS pos=( 4.6333, 2.1916, 1.4235), 0 bonds, id: 5101952304>" is element: "O"
  atom, element))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### Fix overlaps by LAMMPS

In [18]:
# Command (lmp_mpi) might be different (lmp) depend on the installed lammps version
!lmp_mpi -in ../util/mdp_files/in.minimize -log minimize.log

/Users/coquach/Documents/TRUE/TRUE-nanotribology/workflow/single_run/example_simulation
LAMMPS (7 Aug 2019)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:93)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 0) to (50 50 529.69)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  16264 atoms
  scanning bonds ...
  5 = max bonds/atom
  scanning angles ...
  10 = max angles/atom
  scanning dihedrals ...
  17 = max dihedrals/atom
  reading bonds ...
  17216 bonds
  reading angles ...
  33326 angles
  reading dihedrals ...
  48356 dihedrals
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:   0          0          0         
  special bond factors coul: 0          0          0         
  5 = max # of 1-2 neighbors
  7 = max # of 1-3 neighbors
  20 = max # of 1-4 neighbors
  25 = max # of special neighbors
  special bonds CPU = 0.022668 secs
  read_data CPU = 0.31831 secs
Finding 1-2 1-3 1-4 neighbors ...
  spe

    8300    24.424165    684.28341    159415.98    161284.27    6169.2641 
    8400    24.352395     628.5072    159378.17     161187.2    6135.5893 
    8500    24.175472    631.58136     159354.2    161157.73    6113.0607 
    8600    23.936624    461.42572       159355     160976.8    6080.7488 
    8700     23.76489     423.2566    159348.43    160923.74    6053.2772 
    8800    23.649764     371.2783    159322.24    160839.99     6028.488 
    8900    23.614675    307.11184    159285.58    160737.46    6004.5868 
    9000    23.556939    130.16991    159257.81    160529.95    5977.1738 
    9100    23.449584     211.7295    159231.78    160580.27    5950.9283 
    9200    23.511948    59.797876    159186.46    160386.05    5935.0425 
    9300     23.63157    27.149555    159125.04    160297.77    5915.9949 
    9400    23.618161    15.743228    159056.03     160216.7     5889.133 
    9500    23.627772   -230.64886    158991.18    159905.93    5860.1237 
    9600    23.411446   -

#### Energy minimize by GROMACS

In [21]:
# Convert .xtc to .gro
%gmx trjconv -s init.gro -f minimize.xtc -o minimize.gro  -b 1.0 -e 1.0
# Grompp
%gmx grompp -f util/mdp_files/em.mdp -c minimize.gro -p init.top -n init.ndx -o em.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt -ntomp 1 -gpu_id 0000000000000000

'/Users/coquach/Documents/TRUE/TRUE-nanotribology/workflow/single_run/example_simulation'

#### NVT Equilibrate by GROMACS

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/nvt.mdp -c em.gro -p init.top -n init.ndx -o nvt.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -v -deffnm nvt -s em.tpr -cpi em.cpt -ntomp 1 -gpu_id 0000000000000000

#### Compress wih GROMACS

In [1]:
# Grompp
%gmx grompp -f util/mdp_files/compress.mdp -c nvt.gro -p init.top -n init.ndx -o compress.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -v -deffnm compress -s em.tpr -cpi em.cpt -ntomp 1 -gpu_id 0000000000000000

#### Shearing at 5nN

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -s shear_5nN.tpr -deffnm shear_5nN -cpi shear_5nN.cpt -cpo shear_5nN.cpt -noappend -ntomp 1 -gpu_id 000000000000000

#### Shearing at 15nN

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -s shear_15nN.tpr -deffnm shear_15nN -cpi shear_15nN.cpt -cpo shear_15nN.cpt -noappend -ntomp 1 -gpu_id 000000000000000

#### Shearing at 25nN

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -s shear_25nN.tpr -deffnm shear_25nN -cpi shear_25nN.cpt -cpo shear_25nN.cpt -noappend -ntomp 1 -gpu_id 000000000000000

#### Calculating some not important stuff

In [ ]:
# Unwrap stuff

# Run COF and F0 calculation